# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import pandas as pd
import numpy as np
import os
import pickle
from sqlalchemy import create_engine
import re
import nltk
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer, accuracy_score, mean_squared_error, classification_report

nltk.download(['punkt', 'wordnet', 'averaged_perceptron_tagger'])

[nltk_data] Downloading package punkt to /home/matteo/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/matteo/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/matteo/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [2]:
# load data from database
cwd = os.getcwd()
dbwd = cwd.replace('/models','/data/').replace('\\models','\\data\\')
engine = create_engine('sqlite:///'+dbwd+'DisasterResponse.db')
df = pd.read_sql_table('df',engine)
X = df['message']
Y = df.iloc[:,4:]

In [3]:
df.shape

(26216, 40)

In [4]:
print(df.columns.values)

['id' 'message' 'original' 'genre' 'related' 'request' 'offer'
 'aid_related' 'medical_help' 'medical_products' 'search_and_rescue'
 'security' 'military' 'child_alone' 'water' 'food' 'shelter' 'clothing'
 'money' 'missing_people' 'refugees' 'death' 'other_aid'
 'infrastructure_related' 'transport' 'buildings' 'electricity' 'tools'
 'hospitals' 'shops' 'aid_centers' 'other_infrastructure'
 'weather_related' 'floods' 'storm' 'fire' 'earthquake' 'cold'
 'other_weather' 'direct_report']


In [5]:
print(Y.columns.values)

['related' 'request' 'offer' 'aid_related' 'medical_help'
 'medical_products' 'search_and_rescue' 'security' 'military'
 'child_alone' 'water' 'food' 'shelter' 'clothing' 'money'
 'missing_people' 'refugees' 'death' 'other_aid' 'infrastructure_related'
 'transport' 'buildings' 'electricity' 'tools' 'hospitals' 'shops'
 'aid_centers' 'other_infrastructure' 'weather_related' 'floods' 'storm'
 'fire' 'earthquake' 'cold' 'other_weather' 'direct_report']


### 2. Write a tokenization function to process your text data

In [6]:
def tokenize(text):
    url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
    detected_urls = re.findall(url_regex, text)
    for url in detected_urls:
        text = text.replace(url, "urlplaceholder")

    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()

    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)

    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [8]:
def model_pipeline():
    pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', RandomForestClassifier()),
    ])

    return pipeline

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, Y)

model = []
model = model_pipeline()
model.fit(X_train, y_train)

/home/matteo/Documents/py36/lib/python3.6/site-packages/sklearn/ensemble/forest.py:248: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...obs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False))])

In [11]:
model.predict(['Hello there is fire everywhere'])

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.]])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [17]:
y_pred = model.predict(X_test)

In [18]:
overall_accuracy = (y_pred == y_test).mean().mean()
print('Average overall accuracy {}'.format(overall_accuracy))

Average overall accuracy 0.9396212660631337


In [19]:
y_pred = pd.DataFrame(y_pred, columns = y_test.columns)
for column in y_test.columns:
    print('FEATURE: {}\n'.format(column))
    print(classification_report(y_test[column],y_pred[column]))

FEATURE: related

              precision    recall  f1-score   support

           0       0.56      0.48      0.52      1477
           1       0.85      0.89      0.87      5039
           2       0.45      0.26      0.33        38

   micro avg       0.79      0.79      0.79      6554
   macro avg       0.62      0.54      0.57      6554
weighted avg       0.78      0.79      0.79      6554

FEATURE: request

              precision    recall  f1-score   support

           0       0.89      0.98      0.93      5435
           1       0.83      0.40      0.54      1119

   micro avg       0.88      0.88      0.88      6554
   macro avg       0.86      0.69      0.73      6554
weighted avg       0.88      0.88      0.87      6554

FEATURE: offer

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6532
           1       0.00      0.00      0.00        22

   micro avg       1.00      1.00      1.00      6554
   macro avg       0.5

/home/matteo/Documents/py36/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


              precision    recall  f1-score   support

           0       0.99      1.00      0.99      6481
           1       0.00      0.00      0.00        73

   micro avg       0.99      0.99      0.99      6554
   macro avg       0.49      0.50      0.50      6554
weighted avg       0.98      0.99      0.98      6554

FEATURE: shops

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6524
           1       0.00      0.00      0.00        30

   micro avg       1.00      1.00      1.00      6554
   macro avg       0.50      0.50      0.50      6554
weighted avg       0.99      1.00      0.99      6554

FEATURE: aid_centers

              precision    recall  f1-score   support

           0       0.99      1.00      0.99      6473
           1       0.00      0.00      0.00        81

   micro avg       0.99      0.99      0.99      6554
   macro avg       0.49      0.50      0.50      6554
weighted avg       0.98      0.99   

### 6. Improve your model
Use grid search to find better parameters. 

In [12]:
def multi_accuracy_score(y_true, y_pred):
    return (y_true == y_pred).mean().mean()

In [ ]:
model = model_pipeline()

parameters = {
    'vect__ngram_range': ((1, 1), (1, 2)),
    'vect__max_df': (0.75, 1.0),
    'vect__max_features': (None, 5000),
    'tfidf__use_idf': (True, False),
    'clf__n_estimators': [10],
    'clf__min_samples_split': [2, 8],
    'clf__max_depth': [4, None],
    'clf__max_features': [None]
}

scorer = make_scorer(mean_squared_error, greater_is_better = False)

scorer = make_scorer(multi_accuracy_score, greater_is_better = True)

cv = GridSearchCV(model, param_grid=parameters, scoring = scorer,verbose = 2, n_jobs = 1)

cv.fit(X_train, y_train)

/home/matteo/Documents/py36/lib/python3.6/site-packages/sklearn/model_selection/_split.py:1943: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Fitting 3 folds for each of 64 candidates, totalling 192 fits
[CV] clf__max_depth=4, clf__max_features=None, clf__min_samples_split=2, clf__n_estimators=10, tfidf__use_idf=True, vect__max_df=0.75, vect__max_features=None, vect__ngram_range=(1, 1) 
[CV]  clf__max_depth=4, clf__max_features=None, clf__min_samples_split=2, clf__n_estimators=10, tfidf__use_idf=True, vect__max_df=0.75, vect__max_features=None, vect__ngram_range=(1, 1), total=  13.2s
[CV] clf__max_depth=4, clf__max_features=None, clf__min_samples_split=2, clf__n_estimators=10, tfidf__use_idf=True, vect__max_df=0.75, vect__max_features=None, vect__ngram_range=(1, 1) 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   17.9s remaining:    0.0s


[CV]  clf__max_depth=4, clf__max_features=None, clf__min_samples_split=2, clf__n_estimators=10, tfidf__use_idf=True, vect__max_df=0.75, vect__max_features=None, vect__ngram_range=(1, 1), total=  12.9s
[CV] clf__max_depth=4, clf__max_features=None, clf__min_samples_split=2, clf__n_estimators=10, tfidf__use_idf=True, vect__max_df=0.75, vect__max_features=None, vect__ngram_range=(1, 1) 
[CV]  clf__max_depth=4, clf__max_features=None, clf__min_samples_split=2, clf__n_estimators=10, tfidf__use_idf=True, vect__max_df=0.75, vect__max_features=None, vect__ngram_range=(1, 1), total=  12.9s
[CV] clf__max_depth=4, clf__max_features=None, clf__min_samples_split=2, clf__n_estimators=10, tfidf__use_idf=True, vect__max_df=0.75, vect__max_features=None, vect__ngram_range=(1, 2) 
[CV]  clf__max_depth=4, clf__max_features=None, clf__min_samples_split=2, clf__n_estimators=10, tfidf__use_idf=True, vect__max_df=0.75, vect__max_features=None, vect__ngram_range=(1, 2), total=  23.2s
[CV] clf__max_depth=4, cl

[CV]  clf__max_depth=4, clf__max_features=None, clf__min_samples_split=2, clf__n_estimators=10, tfidf__use_idf=True, vect__max_df=1.0, vect__max_features=5000, vect__ngram_range=(1, 2), total=  14.5s
[CV] clf__max_depth=4, clf__max_features=None, clf__min_samples_split=2, clf__n_estimators=10, tfidf__use_idf=False, vect__max_df=0.75, vect__max_features=None, vect__ngram_range=(1, 1) 
[CV]  clf__max_depth=4, clf__max_features=None, clf__min_samples_split=2, clf__n_estimators=10, tfidf__use_idf=False, vect__max_df=0.75, vect__max_features=None, vect__ngram_range=(1, 1), total=  11.3s
[CV] clf__max_depth=4, clf__max_features=None, clf__min_samples_split=2, clf__n_estimators=10, tfidf__use_idf=False, vect__max_df=0.75, vect__max_features=None, vect__ngram_range=(1, 1) 
[CV]  clf__max_depth=4, clf__max_features=None, clf__min_samples_split=2, clf__n_estimators=10, tfidf__use_idf=False, vect__max_df=0.75, vect__max_features=None, vect__ngram_range=(1, 1), total=  11.2s
[CV] clf__max_depth=4,

[CV]  clf__max_depth=4, clf__max_features=None, clf__min_samples_split=2, clf__n_estimators=10, tfidf__use_idf=False, vect__max_df=1.0, vect__max_features=5000, vect__ngram_range=(1, 2), total=  12.8s
[CV] clf__max_depth=4, clf__max_features=None, clf__min_samples_split=2, clf__n_estimators=10, tfidf__use_idf=False, vect__max_df=1.0, vect__max_features=5000, vect__ngram_range=(1, 2) 
[CV]  clf__max_depth=4, clf__max_features=None, clf__min_samples_split=2, clf__n_estimators=10, tfidf__use_idf=False, vect__max_df=1.0, vect__max_features=5000, vect__ngram_range=(1, 2), total=  12.7s
[CV] clf__max_depth=4, clf__max_features=None, clf__min_samples_split=2, clf__n_estimators=10, tfidf__use_idf=False, vect__max_df=1.0, vect__max_features=5000, vect__ngram_range=(1, 2) 
[CV]  clf__max_depth=4, clf__max_features=None, clf__min_samples_split=2, clf__n_estimators=10, tfidf__use_idf=False, vect__max_df=1.0, vect__max_features=5000, vect__ngram_range=(1, 2), total=  12.7s
[CV] clf__max_depth=4, cl

[CV]  clf__max_depth=4, clf__max_features=None, clf__min_samples_split=8, clf__n_estimators=10, tfidf__use_idf=True, vect__max_df=1.0, vect__max_features=5000, vect__ngram_range=(1, 1), total=  12.0s
[CV] clf__max_depth=4, clf__max_features=None, clf__min_samples_split=8, clf__n_estimators=10, tfidf__use_idf=True, vect__max_df=1.0, vect__max_features=5000, vect__ngram_range=(1, 1) 
[CV]  clf__max_depth=4, clf__max_features=None, clf__min_samples_split=8, clf__n_estimators=10, tfidf__use_idf=True, vect__max_df=1.0, vect__max_features=5000, vect__ngram_range=(1, 1), total=  12.0s
[CV] clf__max_depth=4, clf__max_features=None, clf__min_samples_split=8, clf__n_estimators=10, tfidf__use_idf=True, vect__max_df=1.0, vect__max_features=5000, vect__ngram_range=(1, 2) 
[CV]  clf__max_depth=4, clf__max_features=None, clf__min_samples_split=8, clf__n_estimators=10, tfidf__use_idf=True, vect__max_df=1.0, vect__max_features=5000, vect__ngram_range=(1, 2), total=  14.4s
[CV] clf__max_depth=4, clf__ma

[CV]  clf__max_depth=4, clf__max_features=None, clf__min_samples_split=8, clf__n_estimators=10, tfidf__use_idf=False, vect__max_df=1.0, vect__max_features=None, vect__ngram_range=(1, 2), total=  21.0s
[CV] clf__max_depth=4, clf__max_features=None, clf__min_samples_split=8, clf__n_estimators=10, tfidf__use_idf=False, vect__max_df=1.0, vect__max_features=5000, vect__ngram_range=(1, 1) 
[CV]  clf__max_depth=4, clf__max_features=None, clf__min_samples_split=8, clf__n_estimators=10, tfidf__use_idf=False, vect__max_df=1.0, vect__max_features=5000, vect__ngram_range=(1, 1), total=  10.5s
[CV] clf__max_depth=4, clf__max_features=None, clf__min_samples_split=8, clf__n_estimators=10, tfidf__use_idf=False, vect__max_df=1.0, vect__max_features=5000, vect__ngram_range=(1, 1) 
[CV]  clf__max_depth=4, clf__max_features=None, clf__min_samples_split=8, clf__n_estimators=10, tfidf__use_idf=False, vect__max_df=1.0, vect__max_features=5000, vect__ngram_range=(1, 1), total=  10.4s
[CV] clf__max_depth=4, cl

[CV]  clf__max_depth=None, clf__max_features=None, clf__min_samples_split=2, clf__n_estimators=10, tfidf__use_idf=True, vect__max_df=1.0, vect__max_features=None, vect__ngram_range=(1, 1), total= 2.6min
[CV] clf__max_depth=None, clf__max_features=None, clf__min_samples_split=2, clf__n_estimators=10, tfidf__use_idf=True, vect__max_df=1.0, vect__max_features=None, vect__ngram_range=(1, 2) 
[CV]  clf__max_depth=None, clf__max_features=None, clf__min_samples_split=2, clf__n_estimators=10, tfidf__use_idf=True, vect__max_df=1.0, vect__max_features=None, vect__ngram_range=(1, 2), total=10.4min
[CV] clf__max_depth=None, clf__max_features=None, clf__min_samples_split=2, clf__n_estimators=10, tfidf__use_idf=True, vect__max_df=1.0, vect__max_features=None, vect__ngram_range=(1, 2) 
[CV]  clf__max_depth=None, clf__max_features=None, clf__min_samples_split=2, clf__n_estimators=10, tfidf__use_idf=True, vect__max_df=1.0, vect__max_features=None, vect__ngram_range=(1, 2), total= 9.9min
[CV] clf__max_d

[CV]  clf__max_depth=None, clf__max_features=None, clf__min_samples_split=2, clf__n_estimators=10, tfidf__use_idf=False, vect__max_df=0.75, vect__max_features=5000, vect__ngram_range=(1, 2), total= 2.2min
[CV] clf__max_depth=None, clf__max_features=None, clf__min_samples_split=2, clf__n_estimators=10, tfidf__use_idf=False, vect__max_df=1.0, vect__max_features=None, vect__ngram_range=(1, 1) 
[CV]  clf__max_depth=None, clf__max_features=None, clf__min_samples_split=2, clf__n_estimators=10, tfidf__use_idf=False, vect__max_df=1.0, vect__max_features=None, vect__ngram_range=(1, 1), total= 2.5min
[CV] clf__max_depth=None, clf__max_features=None, clf__min_samples_split=2, clf__n_estimators=10, tfidf__use_idf=False, vect__max_df=1.0, vect__max_features=None, vect__ngram_range=(1, 1) 
[CV]  clf__max_depth=None, clf__max_features=None, clf__min_samples_split=2, clf__n_estimators=10, tfidf__use_idf=False, vect__max_df=1.0, vect__max_features=None, vect__ngram_range=(1, 1), total= 2.4min
[CV] clf_

In [64]:
print(cv.best_estimator_.named_steps.clf)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=8,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)


### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [65]:
y_pred_cv = cv.best_estimator_.predict(X_test)
print((y_pred_cv == y_test).mean().mean())

0.9472078120231919


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

In [61]:
# save the model to disk
filename = 'finalized_model.sav'
pickle.dump(model, open(filename, 'wb'))
 
# some time later...
 
# load the model from disk
# loaded_model = pickle.load(open(filename, 'rb'))
# result = loaded_model.score(X_test, Y_test)

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.